In [0]:
%sh
rm -r /dbfs/hyperparam_tune_lab -f
mkdir /dbfs/hyperparam_tune_lab
wget -O /dbfs/hyperparam_tune_lab/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv

--2025-05-21 07:10:34--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9533 (9.3K) [text/plain]
Saving to: ‘/dbfs/hyperparam_tune_lab/penguins.csv’

     0K .........                                             100% 2.19M=0.004s

2025-05-21 07:10:35 (2.19 MB/s) - ‘/dbfs/hyperparam_tune_lab/penguins.csv’ saved [9533/9533]



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
   
# CSV 파일을 Spark DataFrame으로 읽어옵니다. header 옵션은 CSV 파일의 첫 줄을 컬럼 이름으로 사용하도록 합니다.
data = spark.read.format("csv").option("header", "true").load("/hyperparam_tune_lab/penguins.csv")

# 결측값이 있는 행을 제거하고(dropna), 각 컬럼을 적절한 데이터 타입으로 변환합니다.
data = data.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"), # 부리 길이
                          col("CulmenDepth").astype("float"),  # 부리 깊이
                          col("FlipperLength").astype("float"),# 지느러미 길이
                          col("BodyMass").astype("float"),   # 몸무게
                          col("Species").astype("int")       # 펭귄 종 (레이블)
                          )
# 데이터의 20%를 무작위로 샘플링하여 Databricks notebook에 시각적으로 표시합니다.
display(data.sample(0.2))
   
# 데이터를 training set (70%)과 test set (30%)으로 무작위로 분할합니다.
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Island,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
Torgersen,39.3,20.6,190.0,3650.0,0
Torgersen,39.2,19.6,195.0,4675.0,0
Torgersen,38.6,21.2,191.0,3800.0,0
Torgersen,38.7,19.0,195.0,3450.0,0
Torgersen,34.4,18.4,184.0,3325.0,0
Biscoe,35.9,19.2,189.0,3800.0,0
Dream,36.4,17.0,195.0,3325.0,0
Dream,39.8,19.1,184.0,4650.0,0
Dream,36.5,18.0,182.0,3150.0,0
Dream,36.0,18.5,186.0,3100.0,0


Training Rows: 243  Testing Rows: 99


In [0]:
import optuna
import mlflow # 실험을 로깅하고 싶다면 사용합니다.
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
   
def objective(trial):
    # Hyperparameter 값 제안 (maxDepth 및 maxBins):
    # "MaxDepth"라는 이름의 hyperparameter에 대해 0에서 9 사이의 정수 값을 제안합니다.
    max_depth = trial.suggest_int("MaxDepth", 0, 9) 
    # "MaxBins"라는 이름의 hyperparameter에 대해 [10, 20, 30] 리스트 중 하나의 범주형 값을 제안합니다.
    max_bins = trial.suggest_categorical("MaxBins", [10, 20, 30])

    # Pipeline 구성 요소 정의
    cat_feature = "Island" # 범주형 feature 컬럼 이름
    num_features = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"] # 수치형 feature 컬럼 이름 리스트
    
    # StringIndexer: 범주형 문자열 컬럼("Island")을 숫자 인덱스 컬럼("IslandIdx")으로 변환합니다.
    catIndexer = StringIndexer(inputCol=cat_feature, outputCol=cat_feature + "Idx")
    # VectorAssembler: 여러 수치형 feature 컬럼들을 단일 벡터 컬럼("numericFeatures")으로 결합합니다.
    numVector = VectorAssembler(inputCols=num_features, outputCol="numericFeatures")
    # MinMaxScaler: "numericFeatures" 벡터의 값들을 0과 1 사이로 정규화하여 "normalizedFeatures" 컬럼을 만듭니다.
    numScaler = MinMaxScaler(inputCol=numVector.getOutputCol(), outputCol="normalizedFeatures")
    # VectorAssembler: 인덱싱된 범주형 feature와 정규화된 수치형 feature들을 최종 "Features" 벡터 컬럼으로 결합합니다.
    featureVector = VectorAssembler(inputCols=[cat_feature + "Idx", "normalizedFeatures"], outputCol="Features")

    # DecisionTreeClassifier: 의사결정 트리 분류기를 설정합니다.
    # labelCol: 예측 대상 컬럼 (펭귄 종)
    # featuresCol: 학습에 사용될 feature 컬럼
    # maxDepth, maxBins: Optuna가 제안한 hyperparameter 값 사용
    dt = DecisionTreeClassifier(
        labelCol="Species",
        featuresCol="Features",
        maxDepth=max_depth,
        maxBins=max_bins
    )

    # Pipeline: 정의된 모든 변환 단계(Indexer, Assembler, Scaler)와 분류기(dt)를 순서대로 실행하는 파이프라인을 만듭니다.
    pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, dt])
    # 모델 학습: training data(train)를 사용하여 파이프라인을 학습시킵니다.
    model = pipeline.fit(train)

    # 정확도를 사용하여 모델 평가
    # 학습된 모델을 사용하여 test data에 대한 예측을 생성합니다.
    predictions = model.transform(test)
    # MulticlassClassificationEvaluator: 다중 클래스 분류 모델의 성능을 평가합니다.
    # metricName="accuracy": 평가 지표로 정확도를 사용합니다.
    evaluator = MulticlassClassificationEvaluator(
        labelCol="Species",
        predictionCol="prediction", # 모델이 생성한 예측값 컬럼
        metricName="accuracy"
    )
    # 예측 결과에 대한 정확도를 계산합니다.
    accuracy = evaluator.evaluate(predictions)

    # Optuna는 목적 함수를 최소화하므로, 정확도의 음수 값을 반환합니다.
    # 이렇게 하면 Optuna가 정확도를 최대화하는 방향으로 최적화를 수행하게 됩니다.
    return -accuracy

In [0]:
# 5번의 trial로 최적화 실행:
# Optuna study 객체를 생성합니다. study는 최적화 과정을 관리합니다.
study = optuna.create_study() 
# objective 함수를 사용하여 최적화를 시작합니다. n_trials=5는 5번의 다른 hyperparameter 조합을 시도하라는 의미입니다.
study.optimize(objective, n_trials=5)

print("최적화 실행에서 찾은 최상의 매개변수 값:")
# study.best_params는 최적화 과정에서 가장 좋은 성능(가장 낮은 손실 값, 즉 가장 높은 정확도)을 보인 hyperparameter 조합을 반환합니다.
print(study.best_params)

[I 2025-05-21 07:27:16,851] A new study created in memory with name: no-name-c88ccd51-8eea-45ba-aba8-261fee8148b3


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

[I 2025-05-21 07:28:06,243] Trial 0 finished with value: -0.7575757575757576 and parameters: {'MaxDepth': 1, 'MaxBins': 10}. Best is trial 0 with value: -0.7575757575757576.


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

[I 2025-05-21 07:28:48,781] Trial 1 finished with value: -0.9393939393939394 and parameters: {'MaxDepth': 3, 'MaxBins': 10}. Best is trial 1 with value: -0.9393939393939394.


...

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

[I 2025-05-21 07:30:15,865] Trial 3 finished with value: -0.9595959595959596 and parameters: {'MaxDepth': 9, 'MaxBins': 30}. Best is trial 2 with value: -0.9696969696969697.


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

[I 2025-05-21 07:30:58,011] Trial 4 finished with value: -0.7777777777777778 and parameters: {'MaxDepth': 1, 'MaxBins': 30}. Best is trial 2 with value: -0.9696969696969697.


최적화 실행에서 찾은 최상의 매개변수 값:
{'MaxDepth': 6, 'MaxBins': 20}
